# Download Sentinel 2 chips 

## Imports

In [1]:
import geopandas as gpd
import io
import json
import pandas as pd
from pathlib import Path
import planetary_computer
import pprint
import pystac
import pystac_client
import rasterio
import rioxarray
from shapely.geometry import Polygon
import stackstac

## Functions

In [2]:
def get_items(pystac_lient_url: str, 
              intersects: dict,
              datetimes: str, 
              # chip: pd.core.series.Series, 
              cloud_cover_lt: int = 80,
              drop_duplicates: bool = True):

    stac = pystac_client.Client.open(pystac_lient_url)
    search = stac.search(
        intersects=intersects,
        datetime=datetimes,
        collections=["sentinel-2-l2a"],
        limit=1000,  # fetch items in batches of 500
        query={
            "eo:cloud_cover": {"lt": cloud_cover_lt},
            # we only work with small chip areas and only use data from one zone
            # this avoids duplicates while we should not miss any data
            # "proj:epsg": {"eq": int(chip.epsg)},
        },
    )
    candidate_items = list(search.get_items())
    ## scenes db for overview
    ids = []
    geometries = []
    cloud_cover = []

    for item in candidate_items:
        ids.append(item.id)
        # poly = Polygon.from_bounds(*item.bbox)
        assert len(item.geometry["coordinates"]) == 1
        poly = Polygon(item.geometry["coordinates"][0])
        geometries.append(poly)
        cloud_cover.append(item.properties["eo:cloud_cover"])

    scenes = pd.DataFrame(
        {"id": ids, "geometry": geometries, "cloud_cover": cloud_cover}
    )

    # https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/naming-convention
    candidate_scenes = pd.concat(
        [
            scenes,
            scenes.id.str.split("_", expand=True).rename(
                {
                    0: "mission",
                    1: "product_level",
                    2: "datatake_sensing_start",
                    3: "relative_orbit",
                    4: "tile",
                    5: "product_discriminator",
                },
                axis=1,
            ),
        ],
        axis=1,
    )

    candidate_scenes["date"] = pd.to_datetime(
        candidate_scenes["datatake_sensing_start"].str[:8]
    )
    
    candidate_scenes["unique_acquisitions"] = False
    keep = candidate_scenes.groupby(["mission", "date"]).first()["id"]
    candidate_scenes.loc[scenes["id"].isin(keep), "unique_acquisitions"] = True

    if drop_duplicates:
        scenes = candidate_scenes.query("unique_acquisitions")
        items = [it for it in candidate_items if it.id in scenes["id"].values]
        assert len(items) == scenes.shape[0]
    else:
        items = candidate_items
    
    return items, candidate_scenes


## Input data and parameters

In [5]:
# # input data
# copy of a geopandas.GeoDataFrame().to_json()
geojson_chips = '{"type": "FeatureCollection", "features": [{"id": "2", "type": "Feature", "properties": {"chip_id": 2, "easting": 688503.1824475648, "epsg": 32632, "id": "Lerchenauer See", "latitude": 48.19723, "longitude": 11.53678, "maxx": 689460.0, "maxy": 5342280.0, "minx": 687600.0, "miny": 5340420.0, "northing": 5341333.603224352, "tiles": "32UPU", "zone_letter": "U", "zone_number": 32}, "geometry": {"type": "Polygon", "coordinates": [[[11.524234094137505, 48.18928587085897], [11.525055996690533, 48.20600379648934], [11.550065587909714, 48.205451427901956], [11.549235566161816, 48.18873382438354], [11.524234094137505, 48.18928587085897]]]}}, {"id": "3", "type": "Feature", "properties": {"chip_id": 3, "easting": 705439.5861520077, "epsg": 32632, "id": "Kieswerk Ebenhoeh", "latitude": 48.18998, "longitude": 11.76433, "maxx": 706380.0, "maxy": 5342040.0, "minx": 704520.0, "miny": 5340180.0, "northing": 5341111.3411528235, "tiles": "32UPU, 32UQU", "zone_letter": "U", "zone_number": 32}, "geometry": {"type": "Polygon", "coordinates": [[[11.751522831107586, 48.18190724189189], [11.752418453742267, 48.198622128828674], [11.777419839448665, 48.19802037502167], [11.77651610687036, 48.18130583892132], [11.751522831107586, 48.18190724189189]]]}}, {"id": "4", "type": "Feature", "properties": {"chip_id": 4, "easting": 706949.9406789518, "epsg": 32632, "id": "Marzlinger Weiher", "latitude": 48.392441, "longitude": 11.795693, "maxx": 707880.0, "maxy": 5364600.0, "minx": 706020.0, "miny": 5362740.0, "northing": 5363696.487794535, "tiles": "32UPU, 32UQU", "zone_letter": "U", "zone_number": 32}, "geometry": {"type": "Polygon", "coordinates": [[[11.782676865182088, 48.38415047868298], [11.783589070509988, 48.40086433526531], [11.808688737925511, 48.400253896920304], [11.807768333467632, 48.38354039650945], [11.782676865182088, 48.38415047868298]]]}}, {"id": "5", "type": "Feature", "properties": {"chip_id": 5, "easting": 704137.0853263629, "epsg": 32632, "id": "Tegernsee", "latitude": 47.7421, "longitude": 11.72314, "maxx": 705060.0, "maxy": 5292180.0, "minx": 703200.0, "miny": 5290320.0, "northing": 5291228.066208213, "tiles": "32UPU, 32UQU, 32TPT, 32TQT", "zone_letter": "T", "zone_number": 32}, "geometry": {"type": "Polygon", "coordinates": [[[11.710230094604286, 47.734234541181756], [11.711098555200053, 47.750951312207526], [11.735885627074119, 47.75036268068942], [11.735009250336178, 47.73364625232421], [11.710230094604286, 47.734234541181756]]]}}, {"id": "1", "type": "Feature", "properties": {"chip_id": 1, "easting": 321827.8428315255, "epsg": 32633, "id": "Raedlinger Weiher", "latitude": 48.67412, "longitude": 12.5797, "maxx": 322740.0, "maxy": 5394960.0, "minx": 320880.0, "miny": 5393100.0, "northing": 5394057.097320923, "tiles": "32UQU, 32UQV, 33UUP, 33UUQ", "zone_letter": "U", "zone_number": 33}, "geometry": {"type": "Polygon", "coordinates": [[[12.567250916755459, 48.66524646552912], [12.5664454366049, 48.68196415944526], [12.591692294447302, 48.68249501125058], [12.59248943795, 48.66577700713926], [12.567250916755459, 48.66524646552912]]]}}]}'
pprint.pprint(json.loads(geojson_chips))
chips = gpd.read_file(io.StringIO(geojson_chips))
chips

{'features': [{'geometry': {'coordinates': [[[11.524234094137505,
                                              48.18928587085897],
                                             [11.525055996690533,
                                              48.20600379648934],
                                             [11.550065587909714,
                                              48.205451427901956],
                                             [11.549235566161816,
                                              48.18873382438354],
                                             [11.524234094137505,
                                              48.18928587085897]]],
                            'type': 'Polygon'},
               'id': '2',
               'properties': {'chip_id': 2,
                              'easting': 688503.1824475648,
                              'epsg': 32632,
                              'id': 'Lerchenauer See',
                              'latitude': 48.19723,
       

,chip_id,easting,epsg,id,latitude,longitude,maxx,maxy,minx,miny,northing,tiles,zone_letter,zone_number,geometry
0,2,688503.182448,32632,Lerchenauer See,48.197230,11.536780,689460.0,5342280.0,687600.0,5340420.0,5.341334e+06,32UPU,U,32,"POLYGON ((11.52423 48.18929, 11.52506 48.20600..."
1,3,705439.586152,32632,Kieswerk Ebenhoeh,48.189980,11.764330,706380.0,5342040.0,704520.0,5340180.0,5.341111e+06,"32UPU, 32UQU",U,32,"POLYGON ((11.75152 48.18191, 11.75242 48.19862..."
2,4,706949.940679,32632,Marzlinger Weiher,48.392441,11.795693,707880.0,5364600.0,706020.0,5362740.0,5.363696e+06,"32UPU, 32UQU",U,32,"POLYGON ((11.78268 48.38415, 11.78359 48.40086..."
3,5,704137.085326,32632,Tegernsee,47.742100,11.723140,705060.0,5292180.0,703200.0,5290320.0,5.291228e+06,"32UPU, 32UQU, 32TPT, 32TQT",T,32,"POLYGON ((11.71023 47.73423, 11.71110 47.75095..."
4,1,321827.842832,32633,Raedlinger Weiher,48.674120,12.579700,322740.0,5394960.0,320880.0,5393100.0,5.394057e+06,"32UQU, 32UQV, 33UUP, 33UUQ",U,33,"POLYGON ((12.56725 48.66525, 12.56645 48.68196..."


In [7]:
bands = ["B02", "B04", "B8A", "B09", "B10", "B11", "B12", "SCL"]
datetimes = "2016-01-01/2016-01-31"

## Download data of one chips

In [14]:
chip_id, drop_duplicates = 1, False # => EPSG issue since there are different CRS involved 
chip_id, drop_duplicates = 1, True

chip = chips.query(f'chip_id == {chip_id}').iloc[0]
# the following function expects chip to be a series with the following attributes:
# - longitude
# - latitude
# - epsg
# - chip_id
# it only queries data for the given epsg
# it removes duplicates from the items list 
# the duplicates are contained in the candidate_scenes and marked in unique_acquisitions
# see assertions below

# query data intersecting with point - CRS must  be long/lat, epsg:4326  
intersects_point = dict(type="Point", coordinates=[chip.longitude, chip.latitude])
pprint.pprint(intersects_point)

# query data intersecting with area - CRS must  be long/lat, epsg:4326  
intersects_poly = dict(type="Polygon", coordinates=[
    [[lon, lat] for lon, lat in zip(*chip.geometry.exterior.coords.xy)]
])
pprint.pprint(intersects_poly)


items, candidate_scenes = get_items(
    pystac_lient_url="https://planetarycomputer.microsoft.com/api/stac/v1",
    intersects=intersects_poly,
    datetimes=datetimes,
    cloud_cover_lt=80,
    drop_duplicates=drop_duplicates
)

# assert len(items) == candidate_scenes.unique_acquisitions.sum()
# assert len(items) <= candidate_scenes.shape[0]
print(candidate_scenes.query('unique_acquisitions')['tile'].unique)
candidate_scenes # .query('unique_acquisitions')

{'coordinates': [12.5797, 48.67412], 'type': 'Point'}
{'coordinates': [[[12.567250916755459, 48.66524646552912],
                  [12.5664454366049, 48.68196415944526],
                  [12.591692294447302, 48.68249501125058],
                  [12.59248943795, 48.66577700713926],
                  [12.567250916755459, 48.66524646552912]]],
 'type': 'Polygon'}
<bound method Series.unique of 0    T33UUQ
4    T33UUP
6    T33UUQ
8    T33UUP
Name: tile, dtype: object>


,id,geometry,cloud_cover,mission,product_level,datatake_sensing_start,relative_orbit,tile,product_discriminator,date,unique_acquisitions
0,S2A_MSIL2A_20160130T101252_R022_T33UUQ_2021052...,"POLYGON ((12.23093206 49.61959688, 12.28537105...",67.255723,S2A,MSIL2A,20160130T101252,R022,T33UUQ,20210528T002412,2016-01-30,True
1,S2A_MSIL2A_20160130T101252_R022_T33UUP_2021052...,"POLYGON ((12.2806415 48.72091487, 12.33245767 ...",9.164781,S2A,MSIL2A,20160130T101252,R022,T33UUP,20210528T002409,2016-01-30,False
2,S2A_MSIL2A_20160130T101252_R022_T32UQV_2021052...,"POLYGON ((11.76851481 49.61961011, 11.71408666...",62.954427,S2A,MSIL2A,20160130T101252,R022,T32UQV,20210528T002402,2016-01-30,False
3,S2A_MSIL2A_20160130T101252_R022_T32UQU_2021052...,"POLYGON ((11.71881526 48.7209277, 11.66700942 ...",9.544387,S2A,MSIL2A,20160130T101252,R022,T32UQU,20210528T002400,2016-01-30,False
4,S2A_MSIL2A_20160120T101332_R022_T33UUP_2021052...,"POLYGON ((12.2806415 48.72091487, 12.33245767 ...",55.272867,S2A,MSIL2A,20160120T101332,R022,T33UUP,20210527T204834,2016-01-20,True
5,S2A_MSIL2A_20160120T101332_R022_T32UQU_2021052...,"POLYGON ((11.71881526 48.7209277, 11.66700942 ...",58.067293,S2A,MSIL2A,20160120T101332,R022,T32UQU,20210527T204827,2016-01-20,False
6,S2A_MSIL2A_20160117T100342_R122_T33UUQ_2021052...,"POLYGON ((12.23093206 49.61959688, 12.28537105...",71.566601,S2A,MSIL2A,20160117T100342,R122,T33UUQ,20210527T191137,2016-01-17,True
7,S2A_MSIL2A_20160117T100342_R122_T33UUP_2021052...,"POLYGON ((12.2806415 48.72091487, 12.33245767 ...",63.912887,S2A,MSIL2A,20160117T100342,R122,T33UUP,20210527T191137,2016-01-17,False
8,S2A_MSIL2A_20160107T100412_R122_T33UUP_2021052...,"POLYGON ((12.2806415 48.72091487, 12.33245767 ...",53.529648,S2A,MSIL2A,20160107T100412,R122,T33UUP,20210527T111925,2016-01-07,True


In [18]:
# the bounds must be in the CRS of the imagery, so for sentinel 2 some UTM Zone 
# # if the images in the stack belong to more than a single CRS, the target CRS must be specified  
epsg = chip.epsg
bounds = [chip.minx, chip.miny, chip.maxx, chip.maxy]
print(epsg)
print(bounds)

# from the items create a xarray DataArray with dims ('time', 'band', 'y', 'x')
stack = stackstac.stack(
    planetary_computer.sign(pystac.ItemCollection(items)),
    assets=bands,
    # epsg=epsg,
    bounds=bounds,
    )

# epsg is needed here if there are multiple crs in the stack.
# but that causes the following error below or with stack.compute() - not clear why
# CRSError: CRS is invalid: 32633

# currently this notebook only works if all the scenes in the stack are from the same crs

32633
[320880.0, 5393100.0, 322740.0, 5394960.0]


Use the correct data type!

In [19]:
stack

<xarray.DataArray 'stackstac-fd8db016922f4ca3232c786d1b67c5bd' (time: 4,
                                                                band: 7,
                                                                y: 186, x: 186)>
dask.array<fetch_raster_window, shape=(4, 7, 186, 186), dtype=float64, chunksize=(1, 1, 186, 186), chunktype=numpy.ndarray>
Coordinates: (12/43)
  * time                                     (time) datetime64[ns] 2016-01-07...
    id                                       (time) <U54 'S2A_MSIL2A_20160107...
  * band                                     (band) <U3 'B02' 'B04' ... 'SCL'
  * x                                        (x) float64 3.209e+05 ... 3.227e+05
  * y                                        (y) float64 5.395e+06 ... 5.393e+06
    s2:vegetation_percentage                 (time) float64 1.878 0.0 ... 10.6
    ...                                       ...
    title                                    (band) <U37 'Band 2 - Blue - 10m...
    gsd                                      (band) float64 10.0 10.0 ... 20.0
    common_name                              (band) object 'blue' 'red' ... None
    center_wavelength                        (band) object 0.49 0.665 ... None
    full_width_half_max                      (band) object 0.098 0.038 ... None
    epsg                                     int32 32633
Attributes:
    spec:        RasterSpec(epsg=32633, bounds=(320880.0, 5393100.0, 322740.0...
    crs:         epsg:32633
    transform:   | 10.00, 0.00, 320880.00|\n| 0.00,-10.00, 5394960.00|\n| 0.0...
    resolution:  10.0

In [21]:
# the stack is of type float
# to store a band in its original data type 
# we can get the information from the raster itself - if we do not know better

lookup_asset_dtype = {}
item = items[0]
for asset_name, asset in item.assets.items():
    if asset_name not in bands:
        continue
    asset_href = asset.href
    signed_href = planetary_computer.sign(asset_href)

    with rasterio.open(signed_href) as ds:
        lookup_asset_dtype[asset_name] = ds.meta['dtype']
lookup_asset_dtype

{'B02': 'uint16',
 'B04': 'uint16',
 'B09': 'uint16',
 'B11': 'uint16',
 'B12': 'uint16',
 'B8A': 'uint16',
 'SCL': 'uint8'}

In [24]:
basedir_chip = f'chip_id-{chip_id}/'
Path(basedir_chip).mkdir(exist_ok=True, parents=True)

for band in stack.coords['band'].values:
    for time in stack.coords['time'].values:
        print(band, time)
        arr = stack.sel(band=band, time=time)
        filename = basedir_chip + arr.coords['s2:product_uri'].values.tolist().replace('.SAFE', f'_{band}.tif')
        epsg = str(arr.coords['proj:epsg'].values.tolist())
        print(rasterio.crs.CRS.from_epsg(epsg))
        arr.astype(lookup_asset_dtype[band]) \
            .rio.to_raster(
                filename, 
                driver="COG", 
                tiled=False, 
                crs=rasterio.crs.CRS({'init': f'EPSG:{epsg}'}) #rasterio.crs.from_epsg(epsg)
        )


B02 2016-01-07T10:04:12.030000000
EPSG:32633
B02 2016-01-17T10:03:42.030000000
EPSG:32633
B02 2016-01-20T10:13:32.029000000
EPSG:32633
B02 2016-01-30T10:12:52.031000000
EPSG:32633
B04 2016-01-07T10:04:12.030000000
EPSG:32633
B04 2016-01-17T10:03:42.030000000
EPSG:32633
B04 2016-01-20T10:13:32.029000000
EPSG:32633
B04 2016-01-30T10:12:52.031000000
EPSG:32633
B8A 2016-01-07T10:04:12.030000000
EPSG:32633
B8A 2016-01-17T10:03:42.030000000
EPSG:32633
B8A 2016-01-20T10:13:32.029000000
EPSG:32633
B8A 2016-01-30T10:12:52.031000000
EPSG:32633
B09 2016-01-07T10:04:12.030000000
EPSG:32633
B09 2016-01-17T10:03:42.030000000
EPSG:32633
B09 2016-01-20T10:13:32.029000000
EPSG:32633
B09 2016-01-30T10:12:52.031000000
EPSG:32633
B11 2016-01-07T10:04:12.030000000
EPSG:32633
B11 2016-01-17T10:03:42.030000000
EPSG:32633
B11 2016-01-20T10:13:32.029000000
EPSG:32633
B11 2016-01-30T10:12:52.031000000
EPSG:32633
B12 2016-01-07T10:04:12.030000000
EPSG:32633
B12 2016-01-17T10:03:42.030000000
EPSG:32633
B12 2016-0